In [20]:
from typing import (Iterator, Dict)
from datetime import datetime
from requests.exceptions import (HTTPError, ConnectionError)
import requests
import csv


def fetch_climate_data(month: datetime) -> str:
    """
    Fetches daily climate data provided by the WINNIPEG THE FORKS weather station for specified month.
    The province/territory code of WINNIPEG THE FORKS is MB, and its climate code is 5023262, as documented in:
    https://drive.google.com/file/d/1egfzGgzUb0RFu_EE5AYFZtsyXPfZ11y2/view?usp=sharing
    
    Parameters:
        month (datetime): The year and month at which daily climate data is being requested
        
    Returns:
        str: the string data of the csv file that was requested
    """
    
    if month is None:
        raise Exception('Invalid month. Argument must not be None.')
    
    _URL = 'https://dd.weather.gc.ca/climate/observations/daily/csv/'
    PROV_CODE = 'MB'
    CLIMATE_CODE = '5023262'
    DAILY_CONST = 'P1D'
    
    filename = '_'.join(['climate_daily', PROV_CODE, CLIMATE_CODE, month.strftime('%Y-%m'), DAILY_CONST]) + '.csv'

    complete_url = _URL + PROV_CODE + '/' + filename
    
    try:
        response = requests.get(complete_url)
        
        # Raise HTTPError if response.status_code < 400
        response.raise_for_status()
        
    except HTTPError as req_err:
        # This will show the error code and the filename that was not found
        raise req_err
    except ConnectionError:
        raise Exception('Failed to connect to the data source.')
    except Exception as exc:
        raise exc
    
    return response.text


def parse_climate_data(climate_data: str) -> Iterator[Dict]:
    """
    Reads the csv string data line by line and produces a list of dictionaries that contain only the relevant data.
    
    Parameters:
        climate_data (str): the string contents of a csv file
        
    Returns:
        Iterator[Dict]: a list iterator of dictionaries containing only the relevant data as keys.
        Use next(iterator) or a for-loop to process this data.
    """
    
    reader = csv.DictReader(climate_data.strip().split('\n'))
    dataset = []
    
    station_name = None
    longitude = None
    latitude = None
    climate_id = None
    
    for i, line in enumerate(reader):
        data = dict()
        
        if i == 0:
            # These four items do not change in the data file, so parse only once
            station_name = line['Station Name'] if line['Station Name'] != '' else None
            longitude = float(line['Longitude (x)']) if line['Longitude (x)'] != '' else None
            latitude = float(line['Latitude (y)']) if line['Latitude (y)'] != '' else None
            climate_id = int(line['Climate ID']) if line['Climate ID'] != '' else None
        
        data['target_date'] = datetime.strptime(line['Date/Time'], '%Y-%m-%d') if line['Date/Time'] != '' else None
        data['station_name'] = station_name
        data['longitude'] = longitude
        data['latitude'] = latitude
        data['climate_id'] = climate_id
        data['min_temp'] = float(line['Min Temp (°C)']) if line['Min Temp (°C)'] != '' else None
        data['max_temp'] = float(line['Max Temp (°C)']) if line['Max Temp (°C)'] != '' else None
        data['mean_temp'] = float(line['Mean Temp (°C)']) if line['Mean Temp (°C)'] != '' else None
        data['total_precip'] = float(line['Total Precip (mm)']) if line['Total Precip (mm)'] != '' else None

        dataset.append(data)
            
    iterator = iter(dataset)
    
    return iterator


# Uncomment the code below to test the functions

# Test Case 1: month param is valid and exists in data source
month = datetime(2001, 3, 1)
responseText = fetch_climate_data(month)
iterator = parse_climate_data(responseText)
# print(next(iterator))
# print(next(iterator))
# print(next(iterator))
for it in iterator:
    print(it)

{'target_date': datetime.datetime(2001, 3, 1, 0, 0), 'station_name': 'WINNIPEG THE FORKS', 'longitude': -97.13, 'latitude': 49.89, 'climate_id': 5023262, 'min_temp': -6.8, 'max_temp': 3.6, 'mean_temp': -1.6, 'total_precip': 0.0}
{'target_date': datetime.datetime(2001, 3, 2, 0, 0), 'station_name': 'WINNIPEG THE FORKS', 'longitude': -97.13, 'latitude': 49.89, 'climate_id': 5023262, 'min_temp': -4.3, 'max_temp': 5.9, 'mean_temp': 0.8, 'total_precip': 0.0}
{'target_date': datetime.datetime(2001, 3, 3, 0, 0), 'station_name': 'WINNIPEG THE FORKS', 'longitude': -97.13, 'latitude': 49.89, 'climate_id': 5023262, 'min_temp': -9.5, 'max_temp': -1.0, 'mean_temp': -5.3, 'total_precip': 0.0}
{'target_date': datetime.datetime(2001, 3, 4, 0, 0), 'station_name': 'WINNIPEG THE FORKS', 'longitude': -97.13, 'latitude': 49.89, 'climate_id': 5023262, 'min_temp': -14.8, 'max_temp': -3.6, 'mean_temp': -9.2, 'total_precip': 0.0}
{'target_date': datetime.datetime(2001, 3, 5, 0, 0), 'station_name': 'WINNIPEG THE

In [ ]:
# Test Case 2: month param is valid and does not exist in data source
month = datetime(1990, 4, 13)
responseText = fetch_climate_data(month)
iterator = parse_climate_data(responseText)

In [52]:
# Test Case 3: month is None
responseText = fetch_climate_data(None)
iterator = parse_climate_data(responseText)

Exception: Invalid month. Argument must not be None.

In [58]:
import pandas as pd

def parse_climate_data2 ( climate_data : str ):
    data = pd.read_csv(climate_data.strip().split('\n'), sep=',')
    
    return data

In [59]:
month = datetime(2019, 6, 1)

responseText = fetch_climate_data(month)
iterator = parse_climate_data(responseText)

In [60]:
next(iterator)

{'target_date': None,
 'station_name': None,
 'longitude': None,
 'latitude': None,
 'climate_id': None,
 'min_temp': None,
 'max_temp': None,
 'mean_temp': None,
 'total_precip': None}

In [61]:
next(iterator)

{'target_date': None,
 'station_name': None,
 'longitude': None,
 'latitude': None,
 'climate_id': None,
 'min_temp': None,
 'max_temp': None,
 'mean_temp': None,
 'total_precip': None}